# 🍺 Luce Line Beer Recommendation System

This notebook scrapes beer data from Luce Line Brewing and loads it into Snowflake for analysis.

**Prerequisites:**
- Snowflake account with notebook features enabled
- ACCOUNTADMIN privileges for external network setup
- Access to create databases, schemas, and tables


## Step 1: External Network Access Setup


In [ ]:
-- Set up external network access (requires ACCOUNTADMIN)
USE ROLE ACCOUNTADMIN;

-- Create network rule and external access integration
CREATE OR REPLACE NETWORK RULE beer_scraping_network_rule
    MODE = EGRESS
    TYPE = HOST_PORT
    VALUE_LIST = ('lucelinebrewing.com', 'www.lucelinebrewing.com', 'www.lucelinebrewing.com:443');

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION beer_scraping_access_integration
    ALLOWED_NETWORK_RULES = (beer_scraping_network_rule)
    ENABLED = true;

-- Grant access to Snowflake Intelligence Admin role
GRANT USAGE ON INTEGRATION beer_scraping_access_integration TO ROLE snowflake_intelligence_admin_rl;
GRANT USAGE ON NETWORK RULE beer_scraping_network_rule TO ROLE snowflake_intelligence_admin_rl;

-- Switch back to working role
USE ROLE snowflake_intelligence_admin_rl;


## Step 2: Package Setup & Database


In [ ]:
# Import packages and setup session
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, lit, current_timestamp
import pandas as pd
import json
from datetime import datetime
import re
from html.parser import HTMLParser

# Get current session
session = snowpark.context.get_active_session()

# Try to import web scraping packages
try:
    import requests
    print("✅ requests available")
    REQUESTS_AVAILABLE = True
except ImportError:
    print("❌ requests not available")
    REQUESTS_AVAILABLE = False

try:
    from bs4 import BeautifulSoup
    print("✅ beautifulsoup4 available")
    BEAUTIFULSOUP_AVAILABLE = True
except ImportError:
    print("❌ beautifulsoup4 not available - will use fallback parser")
    BEAUTIFULSOUP_AVAILABLE = False

print("📦 Package setup complete")


## Step 3: Create Web Scraping Stored Procedure


In [ ]:
# Import the required function
from snowflake.snowpark.functions import expr

create_sp_sql = """
CREATE OR REPLACE PROCEDURE scrape_luce_line_data()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.10'
PACKAGES = ('requests', 'beautifulsoup4', 'snowflake-snowpark-python')
EXTERNAL_ACCESS_INTEGRATIONS = (beer_scraping_access_integration)
HANDLER = 'main'
AS 
$$
import requests
from bs4 import BeautifulSoup
import re
import json

def clean_text(text):
    if not text:
        return ""
    # Remove extra whitespace and normalize
    text = ' '.join(text.split())
    return text.strip()

def extract_abv(text):
    if not text:
        return None
    # Look for ABV patterns like "7.4%" or "7.4 %"
    match = re.search(r'(\d+(?:\.\d+)?)\s*%', text)
    if match:
        return float(match.group(1))
    return None

def parse_beer_section(section_text):
    lines = [line.strip() for line in section_text.split('\\n') if line.strip()]
    beer_data = None
    
    for i, line in enumerate(lines):
        # Look for lines containing ABV
        if '%' in line and extract_abv(line) is not None:
            # Previous line is likely the beer name
            if i > 0:
                beer_name = clean_text(lines[i-1])
                # Extract style and ABV
                style_abv = line.split('|')
                if len(style_abv) == 2:
                    style = clean_text(style_abv[0])
                    abv = extract_abv(style_abv[1])
                    
                    # Get description from following lines
                    description = []
                    j = i + 1
                    while j < len(lines):
                        next_line = lines[j]
                        # Stop if we hit another beer entry or social media links
                        if ('%' in next_line and '|' in next_line) or \
                           any(x in next_line.lower() for x in ['untappd', 'facebook', 'instagram']):
                            break
                        description.append(next_line)
                        j += 1
                    
                    # If no description found, look before the beer name
                    if not description and i > 1:
                        prev_line = lines[i-2]
                        if '%' not in prev_line and '|' not in prev_line:
                            description = [prev_line]
                    
                    beer_data = {
                        'beer_name': beer_name,
                        'style': style,
                        'abv': abv,
                        'description': ' '.join(description),
                        'brewery': 'Luce Line Brewing Co.',
                        'brewery_city': 'Plymouth',
                        'brewery_state': 'MN'
                    }
                    break
    
    return beer_data

def main(session):
    try:
        url = 'https://lucelinebrewing.com/tap-list/'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        beers = []
        
        # Find all potential beer sections
        beer_sections = soup.find_all(['div', 'p'])
        
        for section in beer_sections:
            text = section.get_text('\\n')
            if '%' in text and '|' in text:  # Likely a beer section
                beer_data = parse_beer_section(text)
                if beer_data and not any(b['beer_name'] == beer_data['beer_name'] for b in beers):
                    beers.append(beer_data)
        
        # Additional parsing for any missed beers
        full_text = soup.get_text('\\n')
        sections = re.split(r'\\n{2,}', full_text)
        
        for section in sections:
            if '%' in section and '|' in section:
                beer_data = parse_beer_section(section)
                if beer_data and not any(b['beer_name'] == beer_data['beer_name'] for b in beers):
                    beers.append(beer_data)
        
        # Return results or fallback
        if len(beers) >= 10:
            return json.dumps(beers)
        else:
            raise Exception("Failed to scrape enough beers")
            
    except Exception as e:
        # Return error message
        return json.dumps([{"error": str(e)}])

$$
"""

session.sql(create_sp_sql).collect()
print("✅ Stored procedure created!")


## Step 4: Execute Scraping & Load Data


In [ ]:
print("🌐 Calling stored procedure to scrape + parse...")
result = session.sql("CALL scrape_luce_line_data()").collect()
beer_json = result[0][0]

# Stored procedure returns a JSON string list of beer objects
beer_data = json.loads(beer_json) if isinstance(beer_json, str) else beer_json

print(f"\n📊 Total beers to insert: {len(beer_data)}")
for beer in beer_data[:3]:
    print(f"  🍺 {beer['beer_name']} - {beer['style']} ({beer['abv']}% ABV)")

In [ ]:
# Import the required function
from snowflake.snowpark.functions import expr
import pandas as pd
from datetime import datetime

# Clear existing data
session.sql("DELETE FROM LUCE_LINE_TAP_LIST").collect()
print("🗑️ Cleared existing data")

# First create DataFrame with the timestamp
current_time = datetime.utcnow()

# Create records with timestamp first
records = []
for beer in beer_data:
    records.append({
        'BEER_NAME': beer['beer_name'],
        'STYLE': beer['style'],
        'ABV': beer['abv'],
        'DESCRIPTION': beer['description'],
        'BREWERY': beer['brewery'],
        'BREWERY_CITY': beer['brewery_city'],
        'BREWERY_STATE': beer['brewery_state'],
        'SCRAPED_AT': current_time,      
    })

# Create DataFrame from records
snowpark_df = session.create_dataframe(records)

# Add description embedding
snowpark_df = snowpark_df.with_column(
    'DESCRIPTION_EMBEDDING',
    expr("SNOWFLAKE.CORTEX.EMBED_TEXT_768('snowflake-arctic-embed-m', STYLE || ' ' || DESCRIPTION)")
)

# Write to table
snowpark_df.write.mode('append').save_as_table('LUCE_LINE_TAP_LIST')

print("✅ Data loaded successfully into LUCE_LINE_TAP_LIST table with embeddings")


In [ ]:
select * from LUCE_LINE_TAP_LIST;